In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np

# 2020년 잠 센서 데이터 처리

In [3]:
sleep_df = pd.read_csv('./2018-2019 data/user_sleep_2019_2018.csv')
sleep_df

,usernum,userId,startDt,endDt,device,sleep_score,total_sleep_time,time_in_bed,waso,wakeupcount,aal,movement_index,fragmentation_index,sleep_frag_index
0,105,user105,2019-12-02 22:38,2019-12-03 18:17,Actigraph,99.240000,70200,70740,540.0,3.0,3.00,0.509,25.000,25.509
1,105,user105,2019-12-08 13:02,2019-12-09 07:59,Actigraph,92.700000,63240,68220,4980.0,27.0,3.07,5.101,21.429,26.530
2,105,user105,2019-12-09 08:11,2019-12-10 07:56,Actigraph,97.050000,82980,85500,2520.0,16.0,2.63,2.386,0.000,2.386
3,105,user105,2019-12-10 10:22,2019-12-11 04:56,Actigraph,98.030000,65520,66840,1320.0,8.0,2.75,2.065,11.111,13.176
4,105,user105,2019-12-11 05:15,2019-12-11 07:58,Actigraph,83.440000,8160,9780,1620.0,10.0,2.70,11.656,18.182,29.838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,26,user26,2018-11-23 02:49,2018-11-23 08:41,Fitbit,90.615385,17670,19500,NaN,NaN,NaN,NaN,NaN,NaN
732,26,user26,2018-11-24 04:00,2018-11-24 17:34,Fitbit,87.500000,34440,39360,NaN,NaN,NaN,NaN,NaN,NaN
733,26,user26,2018-11-25 04:10,2018-11-25 13:34,Fitbit,90.024135,33570,37290,NaN,NaN,NaN,NaN,NaN,NaN
734,26,user26,2018-11-26 02:21,2018-11-26 12:12,Fitbit,87.614679,17190,19620,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
sleep_df = pd.read_csv('./2018-2019 data/user_sleep_2019_2018.csv')
# 지역은 상관 없으므로 삭제
sleep_df = sleep_df.drop(['usernum','device'],axis=1)
# 한국 시간 기준으로 설정하기 위해 timedelta(hours=9)를 진행함
sleep_df['startDt'] = pd.to_datetime(sleep_df['startDt']) + timedelta(hours=9)
sleep_df['endDt'] = pd.to_datetime(sleep_df['endDt']) + timedelta(hours=9)

#start가 0~18시이면 이전날을 date에 삽입, 그게 아니라면 당일날을 date에 삽입
date_tmp = pd.DataFrame(columns=['date'])
for i in range(len(sleep_df)):
    if 0<=sleep_df['startDt'][i].hour<=18:
        date_tmp.loc[i,'date'] = pd.to_datetime(sleep_df.loc[i,'startDt']) - timedelta(days=1)
    else:
        date_tmp.loc[i,'date'] = pd.to_datetime(sleep_df.loc[i,'startDt'])
date_tmp['date'] = date_tmp['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

sleep_df=pd.concat([sleep_df,date_tmp],axis=1)

#당일 전 데이터를 당일 데이터에 붙여줘야함
tmp = pd.DataFrame(columns=["Pre_"+sleep_df.keys()[1:-1]])
a = pd.to_datetime(sleep_df['date']) -  timedelta(days=1)
a = a.apply(lambda x: x.strftime('%Y-%m-%d'))
tmp['Pre_date']= a

# 이전 데이트의 값을 tmp에 누적시키는 과정
pre_date = 0
for i in range(1,len(sleep_df)):
    if ((sleep_df['date'][i-1]) == (tmp['Pre_date'].iloc[i])).bool() or ((tmp['Pre_date'].iloc[i]) == pre_date).bool():
        tmp.iloc[i,:-1] = sleep_df.iloc[i-1,1:-1]
    pre_date = tmp['Pre_date'].iloc[i]

sleep_df=pd.concat([sleep_df,tmp],axis=1)
sleep_df

,userId,startDt,endDt,sleep_score,total_sleep_time,time_in_bed,waso,wakeupcount,aal,movement_index,...,"(Pre_sleep_score,)","(Pre_total_sleep_time,)","(Pre_time_in_bed,)","(Pre_waso,)","(Pre_wakeupcount,)","(Pre_aal,)","(Pre_movement_index,)","(Pre_fragmentation_index,)","(Pre_sleep_frag_index,)","(Pre_date,)"
0,user105,2019-12-03 07:38:00,2019-12-04 03:17:00,99.240000,70200,70740,540.0,3.0,3.00,0.509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01
1,user105,2019-12-08 22:02:00,2019-12-09 16:59:00,92.700000,63240,68220,4980.0,27.0,3.07,5.101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-07
2,user105,2019-12-09 17:11:00,2019-12-10 16:56:00,97.050000,82980,85500,2520.0,16.0,2.63,2.386,...,92.7,63240,68220,4980.0,27.0,3.07,5.101,21.429,26.53,2019-12-07
3,user105,2019-12-10 19:22:00,2019-12-11 13:56:00,98.030000,65520,66840,1320.0,8.0,2.75,2.065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-09
4,user105,2019-12-11 14:15:00,2019-12-11 16:58:00,83.440000,8160,9780,1620.0,10.0,2.70,11.656,...,98.03,65520,66840,1320.0,8.0,2.75,2.065,11.111,13.176,2019-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,user26,2018-11-23 11:49:00,2018-11-23 17:41:00,90.615385,17670,19500,NaN,NaN,NaN,NaN,...,87.5,18060,20640,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-21
732,user26,2018-11-24 13:00:00,2018-11-25 02:34:00,87.500000,34440,39360,NaN,NaN,NaN,NaN,...,90.615385,17670,19500,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-22
733,user26,2018-11-25 13:10:00,2018-11-25 22:34:00,90.024135,33570,37290,NaN,NaN,NaN,NaN,...,87.5,34440,39360,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-23
734,user26,2018-11-26 11:21:00,2018-11-26 21:12:00,87.614679,17190,19620,NaN,NaN,NaN,NaN,...,90.024135,33570,37290,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-24


In [5]:
sleep_df.to_csv('./2018-2019 data/update_user_sleep_2019_2018.csv',index=False)

# 2020년 잠 설문조사 데이터 처리

In [8]:
survey_df = pd.read_csv('./2018-2019 data/user_survey_2019_2018.csv')
a = pd.to_datetime(survey_df['inputDt'])
a = a.apply(lambda x: x.strftime('%Y-%m-%d'))
survey_df['date'] = a

#am 과 pm을 결합하는 과정
am_df = survey_df[survey_df['amPm']=='am'].reset_index(drop = True)
am_df=am_df.drop(['pmEmotion', 'pmStress', 'ifUnusual', 'breakfast', 'lunch', 'dinner', 
            'lateSnack', 'amCaffeine', 'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 
            'alcohol', 'aAmount','inputDt','usernum','amPm'],axis=1)
pm_df = survey_df[survey_df['amPm']=='pm'].reset_index(drop = True)
pm_df=pm_df.drop(['sleep','sleepProblem','dream','amCondition','amEmotion','inputDt','usernum','amPm'],axis=1)
survey_df=am_df.merge(pm_df,how = 'outer', on=['userId','date'])

# 카페인의 여부와 알코올의 여부를 labeling 하는 과정
survey_df['caffeine'] = np.NaN
survey_df.loc[survey_df['amCaffeine'].isnull() == False,'caffeine'] = 1
survey_df['caffeine'].fillna(0,inplace=True)
survey_df.loc[survey_df['pmCaffeine'].isnull() == False,'caffeine'] = 1
survey_df['caffeine'].fillna(0,inplace=True)
survey_df['caffeine'] = survey_df['caffeine'].astype(int)
survey_df.loc[survey_df['alcohol'].isnull() == False,'alcohol'] = 1
survey_df['alcohol'].fillna(0,inplace=True)
survey_df=survey_df.drop(['amCaffeine','amCaffAmount','pmCaffeine','pmCaffAmount','aAmount'],axis=1)

#date feature 위치 변경
survey_df = survey_df[['userId','date', 'sleep', 'sleepProblem', 'dream', 'amCondition', 
                       'amEmotion','pmEmotion', 'pmStress', 'ifUnusual', 'breakfast', 'lunch',
                       'dinner', 'lateSnack', 'alcohol', 'caffeine']]

# 당일 이후 데이터를 당일 데이터에 붙히는 과정
tmp = pd.DataFrame(columns=["Post_"+survey_df.keys()[2:7]])
a = pd.to_datetime(survey_df['date']) +  timedelta(days=1)
a = a.apply(lambda x: x.strftime('%Y-%m-%d'))
tmp['Post_date']= a

post_date = 0
for i in range(len(survey_df)-1):
    if ((survey_df['date'][i+1]) == (tmp['Post_date'].iloc[i])).bool() or ((tmp['Post_date'].iloc[i]) == post_date).bool():
        tmp.iloc[i,:-1] = survey_df.iloc[i+1,2:7]
    post_date = tmp['Post_date'].iloc[i]

survey_df=pd.concat([survey_df,tmp],axis=1)

survey_df

,userId,date,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,ifUnusual,...,dinner,lateSnack,alcohol,caffeine,"(Post_sleep,)","(Post_sleepProblem,)","(Post_dream,)","(Post_amCondition,)","(Post_amEmotion,)","(Post_date,)"
0,user117,2020-01-18,4.0,2.0,4.0,3.0,3.0,3.0,4.0,1.0,...,3.0,4.0,0,1,4.0,0.0,4.0,3.0,3.0,2020-01-19
1,user117,2020-01-19,4.0,0.0,4.0,3.0,3.0,3.0,4.0,2.0,...,3.0,4.0,0,1,5.0,5.0,4.0,2.0,3.0,2020-01-20
2,user117,2020-01-20,5.0,5.0,4.0,2.0,3.0,4.0,4.0,2.0,...,3.0,3.0,0,0,3.0,2.0,4.0,2.0,3.0,2020-01-21
3,user117,2020-01-21,3.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,...,3.0,4.0,0,0,4.0,5.0,4.0,2.0,2.0,2020-01-22
4,user117,2020-01-22,4.0,5.0,4.0,2.0,2.0,3.0,2.0,2.0,...,3.0,4.0,0,1,5.0,0.0,4.0,2.0,3.0,2020-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,user16,2018-11-09,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0,...,2.0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2018-11-10
719,user8,2018-10-31,NaN,NaN,NaN,NaN,NaN,4.0,2.0,2.0,...,3.0,4.0,0,1,NaN,NaN,NaN,NaN,NaN,2018-11-01
720,user8,2018-11-13,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2.0,...,2.0,4.0,0,1,NaN,NaN,NaN,NaN,NaN,2018-11-14
721,user19,2018-11-24,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.0,...,3.0,3.0,1,0,NaN,NaN,NaN,NaN,NaN,2018-11-25


# 설문조사와 잠 데이터 userId, date 기준으로 병합

In [9]:
survey_sleep_df = survey_df.merge(sleep_df, how = 'outer', on=['userId','date'])
survey_sleep_df

,userId,date,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,ifUnusual,...,"(Pre_sleep_score,)","(Pre_total_sleep_time,)","(Pre_time_in_bed,)","(Pre_waso,)","(Pre_wakeupcount,)","(Pre_aal,)","(Pre_movement_index,)","(Pre_fragmentation_index,)","(Pre_sleep_frag_index,)","(Pre_date,)"
0,user117,2020-01-18,4.0,2.0,4.0,3.0,3.0,3.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,user117,2020-01-19,4.0,0.0,4.0,3.0,3.0,3.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,user117,2020-01-20,5.0,5.0,4.0,2.0,3.0,4.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,user117,2020-01-21,3.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,user117,2020-01-22,4.0,5.0,4.0,2.0,2.0,3.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,user2,2018-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-23
970,user28,2018-11-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-11
971,user9,2018-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-30
972,user26,2018-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-13


# User마다 데이터 갯수가 몇개인지 확인

In [10]:
survey_sleep_df['userId'].value_counts()

user104    47
user120    43
user106    36
user115    35
user113    35
user118    35
user111    33
user107    33
user110    30
user108    29
user109    28
user114    27
user119    27
user112    22
user105    21
user117    17
user27     17
user102    16
user116    16
user4      16
user10     15
user6      15
user22     15
user5      15
user7      15
user28     15
user23     15
user26     15
user20     15
user29     14
user9      14
user19     14
user8      14
user103    14
user14     14
user21     14
user25     14
user24     14
user3      14
user30     14
user1      14
user15     14
user11     14
user13     13
user18     13
user101    12
user12     12
user2      12
user17     11
user16      7
Name: userId, dtype: int64

In [11]:
survey_sleep_df=survey_sleep_df.sort_values(['userId','date'],axis=0).reset_index(drop=True)
survey_sleep_df

,userId,date,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,pmStress,ifUnusual,...,"(Pre_sleep_score,)","(Pre_total_sleep_time,)","(Pre_time_in_bed,)","(Pre_waso,)","(Pre_wakeupcount,)","(Pre_aal,)","(Pre_movement_index,)","(Pre_fragmentation_index,)","(Pre_sleep_frag_index,)","(Pre_date,)"
0,user1,2018-11-15,3.0,0.0,4.0,2.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-14
1,user1,2018-11-16,4.0,0.0,4.0,2.0,2.0,3.0,1.0,2.0,...,91.440081,27240,29790,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-15
2,user1,2018-11-17,3.0,3.0,2.0,4.0,5.0,5.0,4.0,2.0,...,90.800478,22800,25110,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-16
3,user1,2018-11-18,3.0,0.0,4.0,3.0,4.0,3.0,2.0,2.0,...,91.984231,21000,22830,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-17
4,user1,2018-11-19,4.0,6.0,4.0,2.0,2.0,4.0,2.0,2.0,...,88.482385,19590,22140,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,user9,2018-11-09,3.0,0.0,4.0,3.0,3.0,4.0,4.0,2.0,...,90.512541,24900,27510,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-08
970,user9,2018-11-10,3.0,0.0,4.0,2.0,3.0,4.0,4.0,2.0,...,89.662677,24720,27570,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-09
971,user9,2018-11-11,4.0,4.0,4.0,3.0,3.0,4.0,4.0,2.0,...,90.070368,34560,38370,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-10
972,user9,2018-11-12,3.0,0.0,2.0,3.0,3.0,3.0,3.0,2.0,...,87.569989,23460,26790,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-11


# csv 파일로 내보내기

In [12]:
survey_sleep_df.to_csv('./2018-2019 data/2018-2019_survey_sleep.csv',index=False)